In [ ]:
import numpy as np
import pandas as pd
import os
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

In [ ]:
import time
import logging
time_now = int(time.time())
logger = logging.getLogger('')
logging.basicConfig(filename=f'portfolio_analysys.{time_now}.log', encoding='utf-8', level=logging.DEBUG)

In [ ]:
df = pd.read_csv('nikkei_mid_small_cap-i5r5ma50-1719447843.csv', parse_dates=['date'])

In [ ]:
# Loop dataframe to get portfolio result

# TODO: import the df to save time rerunning the code

def portfolio_analysis(
        _data: pd.DataFrame,
        initial_capital = 200000,
        threshold = 0.7,
        logging = False,
    ):
    if logging:
        logger.info(f'-----------------------')
        logger.info(f'-----Start Logging-----')
        logger.info(f'-----------------------')
    # meta = []
    dates = []
    capitals = []
    current = initial_capital
    for index, row in tqdm(_data.iterrows()):
        s_return = 0
        s_count = 0
        l_return = 0
        l_count = 0
        for i, _v in row.items():
            if (i == 'date'):
                if logging:
                    logger.info(f'')
                    logger.info(f'---{_v}---')
                continue
            if (type(_v) is not str):
                continue
            # parse array-format-string to actual array
            v = _v[1:-1].split(',')
            if (type(v) is list):
                # short
                if float(v[0]) >= threshold:
                    if logging:
                        logger.info(f'{i} is included in [Short] - p:{v[0]}, r:{v[3]} ')
                    s_return = (s_return * s_count + float(v[3])) / (s_count + 1)
                    s_count += 1
                # long
                if float(v[2]) >= threshold:
                    if logging:
                        logger.info(f'{i} is included in [Long] - p:{v[2]}, r:{v[3]} ')
                    l_return = (l_return * l_count + float(v[3])) / (l_count + 1)
                    l_count += 1
        # skip if no short or long
        if (s_count == 0 or l_count == 0):
            if logging:
                logger.info(f'Not doing anything: l/s count: {l_count} / {s_count}')
                logger.info(f'Final Capital: {current}')
            dates.append(row['date'])
            capitals.append(current)
            continue
        l_capital = (current / 2) * (1 + l_return)
        s_capital = (current / 2) * (1 - s_return)
        if logging:
            logger.info(f'current: {current}')
            logger.info(f'l_return: {l_return}')
            logger.info(f's_return: {s_return}')
            logger.info(f'l_capital: {l_capital}')
            logger.info(f's_capital: {s_capital}')
        current = l_capital + s_capital
        # print('doing trade on ' + str(row['date']))
        # print('current: ' + str(current))
        dates.append(row['date'])
        capitals.append(current)
        if logging:
            logger.info(f'Final Capital: {current}')

    data = {
        'date': np.array(dates),
        'capital': np.array(capitals),
    }
    return data

portfolio_result = portfolio_analysis(df, 200000, 0.5, True)

In [ ]:
ts = pd.Series(portfolio_result['capital'], index=portfolio_result['date'])
ts.plot();

In [ ]:
pd.DataFrame.from_dict(portfolio_result).to_csv('portfolio_result.csv')